# Assignment 2.3: Crime trends reports SFPD in Bokeh

#### Social Data Analysis and Visualization 
#### DTU -  2021

### Import DataSet : [Police Department Incident Reports: Historical 2003 to May 2018](https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-Historical-2003/tmnf-yvry)

In [1]:
import pandas as pd 
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import folium

`.csv`

In [ ]:
crime_data = pd.read_csv("../data/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv",index_col = 0,low_memory=False)

crime_data.drop(crime_data.tail(5).index,inplace=True) # Remove final error

crime_data.head()

`.tsv`

In [5]:
crime_data = pd.read_table("../data/Police_Department_Incident_Reports__Historical_2003_to_May_2018.tsv",index_col = 0,low_memory=False)

crime_data.drop(crime_data.tail(5).index,inplace=True) # Remove final error

crime_data.head()

/usr/local/Cellar/python@3.7/3.7.10_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.
/usr/local/Cellar/python@3.7/3.7.10_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,IncidntNum,Incident Code,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,...,Fix It Zones as of 2017-11-06 2 2,DELETE - HSOC Zones 2 2,Fix It Zones as of 2018-02-07 2 2,"CBD, BID and GBD Boundaries as of 2017 2 2","Areas of Vulnerability, 2016 2 2",Central Market/Tenderloin Boundary 2 2,Central Market/Tenderloin Boundary Polygon - Updated 2 2,HSOC Zones as of 2018-06-05 2 2,OWED Public Spaces 2 2,Neighborhoods 2
PdId,,,,,,,,,,,,,,,,,,,,,
3114751606302,31147516,6302,LARCENY/THEFT,PETTY THEFT FROM A BUILDING,Sunday,09/28/2003,10:00,SOUTHERN,NONE,1ST ST / BRANNAN ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5069701104134,50697011,4134,ASSAULT,BATTERY,Wednesday,06/22/2005,12:20,NORTHERN,NONE,1200 Block of EDDY ST,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,97.0
6074729204104,60747292,4104,ASSAULT,ASSAULT,Saturday,07/15/2006,00:55,CENTRAL,NONE,1600 Block of POWELL ST,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,106.0
7103536315201,71035363,15201,ASSAULT,STALKING,Tuesday,09/25/2007,00:01,TARAVAL,NONE,400 Block of ULLOA ST,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,49.0
11082415274000,110824152,74000,MISSING PERSON,MISSING ADULT,Saturday,09/24/2011,11:00,TARAVAL,LOCATED,3200 Block of SAN JOSE AV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
focuscrimes = set(['WEAPON LAWS', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUNKENNESS', 'DRUG/NARCOTIC', 'TRESPASS', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'DISORDERLY CONDUCT'])

> Recreate the results from *Week 2* as an **interactive Boeak visualisation**. To complete the exercise, we follow the steps stated in the exercise to create the dataviz version.

## 1. Data Prep

> A key step is to set up the data rightfor the dataviz, following the workflow:

> **1.** Take the data for the **period 2010-2018** and group it by hour-of-the-day

Transform *Date* into a time format.

In [7]:
crime_data["Date"]=pd.to_datetime(crime_data["Date"], format='%m/%d/%Y')

Select all the entries between the indicated period.

In [8]:
crime_data_10_18 = crime_data[crime_data["Date"].between('01/01/2010', '31/12/2018')]


Group data by hour-of-the-day.

In [9]:
crime_data["Hour"]= pd.to_datetime(crime_data["Time"], format='%H:%M').dt.hour 
crime_data["HourInt"]= [f'{str(h)}h-{str(h+1)}h' for h in crime_data["Hour"]]

crime_count_by_hour_and_category=crime_data[crime_data["Category"].isin(focuscrimes)
                                           ].groupby(["Category", "Hour", "HourInt"]).count()["IncidntNum"].reset_index()
crime_count_by_hour_and_category.head()

,Category,Hour,HourInt,IncidntNum
0,ASSAULT,0,0h-1h,9048
1,ASSAULT,1,1h-2h,8233
2,ASSAULT,2,2h-3h,7387
3,ASSAULT,3,3h-4h,3627
4,ASSAULT,4,4h-5h,2284


> **2.** **Normalize data** to be able to easily compare how the distribution of crimes differ from each other, not absolute numbers
>> To normalise data for within a crime category we devide the count for each hour by the total number of this crime type.

Incident number **total sum by category**.

In [10]:
crime_count_by_category=crime_data[crime_data["Category"].isin(focuscrimes)
                                           ].groupby(["Category"]).count()["IncidntNum"].reset_index()
crime_count_by_category.head()

,Category,IncidntNum
0,ASSAULT,167041
1,BURGLARY,91067
2,DISORDERLY CONDUCT,9932
3,DRIVING UNDER THE INFLUENCE,5652
4,DRUG/NARCOTIC,117820


**Divide** each entry in the table grouped **by hour-of-the-day** by its total incident number **by category** (one-line `lambda` *functional programing*), storing it into a new column `NormalizedIncidntNum`.

In [11]:
crime_count_by_hour_and_category['NormalizedIncidntNum'] = crime_count_by_hour_and_category.apply(lambda x: x.IncidntNum/crime_count_by_category[crime_count_by_category["Category"]==x.Category].IncidntNum.values[0], axis=1)

Reshape the normalized data `NormalizedIncidntNum` into a table with **`crime-type` as columns and `hours` as rows**.

In [12]:
crime_count_by_hour_and_category = crime_count_by_hour_and_category.pivot(index=crime_count_by_hour_and_category.Hour, columns='Category')['NormalizedIncidntNum']

In [13]:
crime_count_by_hour_and_category.head()

Category,ASSAULT,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,LARCENY/THEFT,PROSTITUTION,ROBBERY,STOLEN PROPERTY,TRESPASS,VANDALISM,VEHICLE THEFT,WEAPON LAWS
Hour,,,,,,,,,,,,,,
0,0.054166,0.038236,0.052255,0.125796,0.031565,0.084631,0.039960,0.081813,0.054014,0.047773,0.029645,0.056443,0.037630,0.053466
1,0.049287,0.024795,0.038462,0.114827,0.019292,0.079816,0.024648,0.063511,0.055740,0.033712,0.020631,0.039515,0.025715,0.039945
2,0.044223,0.026750,0.034535,0.115357,0.015643,0.067930,0.016049,0.047088,0.058219,0.028210,0.023497,0.037318,0.020352,0.035469
3,0.021713,0.027617,0.021748,0.051132,0.010601,0.026434,0.010082,0.032907,0.035085,0.022096,0.019225,0.025140,0.012652,0.022139
4,0.013673,0.024476,0.016110,0.018931,0.007724,0.013525,0.006561,0.028180,0.022711,0.019651,0.014327,0.016763,0.009768,0.014902


## 2. Bokeh Visualization

In [14]:
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure, output_file, show
from bokeh.palettes import turbo

Bokeh view into the *JupyterNotebook*:

In [15]:
from bokeh.io import push_notebook, show, output_notebook
output_notebook()

Loading BokehJS ...

> We follow the Bokeh guide for categorical data, in order to plot our ready preprocessed data:

> **1.** Convert our `Pandas Dataframe` to `Bokeh ColumnDataSource`: 

In [16]:
source = ColumnDataSource(crime_count_by_hour_and_category)

> **2.** Create an empty figure, in which we will input our data later on: 

In [17]:
hours = [str(e) for e in list(crime_count_by_hour_and_category.index)]

In [18]:
p = figure(x_range=FactorRange(factors=hours), plot_height=500, width=1000, title="Crimes per hour",
           toolbar_location=None, tools="", x_axis_label='Hour of the day', y_axis_label='Relative Fequency')


> **3.** Add the bars, with `vbar`: 

In [19]:
bar ={} # to store vbars
palette = turbo(len(focuscrimes)) # selected palette of the size of types of crimes

for indx,i in enumerate(focuscrimes):
    bar[i] = p.vbar(x='Hour',  top=i, source= source, 
                 ### create a vbar for each focuscrime
                 legend_label=i,  color = palette[indx], muted_alpha=0.1, muted =True) 

    p.legend.click_policy="mute" ### assigns the click policy


> **3.** Set **legend** out-right of the plot: 

In [20]:
p.legend.location = "top_left"
p.add_layout(p.legend[0], 'right')

> **4.** Display the interactive figure, within  the `jupyter notebook`: 

In [21]:
show(p, notebook_handle=True);

## References

**[1]** Bokeh Docs [https://docs.bokeh.org/]

**[2]** Interactive bokeh in Jupyter notebook [https://stackoverflow.com/questions/53217654/how-to-get-interactive-bokeh-in-jupyter-notebook]